# Caso práctico de resumen (datos de créditos bancarios)

In [1]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn import metrics

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## 1. Carga de datos, análisis exploratorio y preproceso

In [2]:
# Cargamos los datos del fichero .csv

credit_g = pd.read_csv('./datos/credit-g.csv')

In [3]:
# Información general del contenido de los datos

credit_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   checking_status         1000 non-null   object
 1   duration                1000 non-null   int64 
 2   credit_history          1000 non-null   object
 3   purpose                 1000 non-null   object
 4   credit_amount           1000 non-null   int64 
 5   savings_status          1000 non-null   object
 6   employment              1000 non-null   object
 7   installment_commitment  1000 non-null   int64 
 8   personal_status         1000 non-null   object
 9   other_parties           1000 non-null   object
 10  residence_since         1000 non-null   int64 
 11  property_magnitude      1000 non-null   object
 12  age                     1000 non-null   int64 
 13  other_payment_plans     1000 non-null   object
 14  housing                 1000 non-null   object
 15  exist

In [4]:
credit_g.describe()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000


In [5]:
credit_g.describe(include = 'object')

,checking_status,credit_history,purpose,savings_status,employment,personal_status,other_parties,property_magnitude,other_payment_plans,housing,job,own_telephone,foreign_worker,class
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,4,5,10,5,5,4,3,4,3,3,4,2,2,2
top,'no checking','existing paid',radio/tv,'<100','1<=X<4','male single',none,car,none,own,skilled,none,yes,good
freq,394,530,280,603,339,548,907,332,814,713,630,596,963,700


In [6]:
# Cambiamos las columnas con pocos valores posibles a tipo categórico

columnas_num = ['duration', 'credit_amount', 'age']
columnas_cat = credit_g.select_dtypes(include = 'object').columns.to_list() + \
               ['installment_commitment', 'residence_since', 'existing_credits', 'num_dependents']

for field in columnas_cat:
    credit_g[field] = credit_g[field].astype('category')

In [7]:
credit_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   checking_status         1000 non-null   category
 1   duration                1000 non-null   int64   
 2   credit_history          1000 non-null   category
 3   purpose                 1000 non-null   category
 4   credit_amount           1000 non-null   int64   
 5   savings_status          1000 non-null   category
 6   employment              1000 non-null   category
 7   installment_commitment  1000 non-null   category
 8   personal_status         1000 non-null   category
 9   other_parties           1000 non-null   category
 10  residence_since         1000 non-null   category
 11  property_magnitude      1000 non-null   category
 12  age                     1000 non-null   int64   
 13  other_payment_plans     1000 non-null   category
 14  housing                 1

## 2. Análisis de la capacidad predictiva de cada variable

In [8]:
# Podemos hacer unos boxplot para ver la distribución de valores de las variables continuas para
# los casos de buen crédito y mal crédito.
# Como se puede ver en los gráficos, los prestamos de mayor duración y de mayor cantidad tienen un
# riesgo mayor (¡en este caso!), y cuanto menor edad, más probabilidad de mal crédito

alt.Chart(credit_g).mark_boxplot().encode(y = alt.X('class:N', title = None), 
                                         x = alt.Y(alt.repeat("row"), type='quantitative'),
                                         color = alt.Color('class:N', legend = None))\
                                  .repeat(row=columnas_num)\
                                  .resolve_scale(x = 'independent')

alt.RepeatChart(...)

In [9]:
# Para las variables categóricas, una opción es mostrar el porcentaje de mal crédito para cada uno
# de los valores posibles de cada una de esas variables
# Como se ve en los gráficos, no hay ninguna variable individual con gran capacidad predictiva,
# aunque algunas parecen dar buenas pistas

def get_data_chunk(field):
    return credit_g.groupby(field)['class'].apply(lambda x: (x == 'bad').mean())\
                   .rename('percentage bad')\
                   .reset_index()\
                   .rename(columns = {field : 'value'})\
                   .assign(var = field)

mal_credito_por_valor = pd.concat([get_data_chunk(field) for field in columnas_cat if field != 'class'])

alt.Chart(mal_credito_por_valor, title = 'Porcentaje de casos con mal crédito').mark_bar()\
                        .encode( x = 'percentage bad:Q', y = alt.Y('value:N', title = None), row = 'var:N')\
                        .resolve_scale(y = 'independent')

alt.Chart(...)

In [10]:
# La variable 'foreign_worker' parece tener bastante capadidad predictiva, pero si hacemos un gráfico para sus distintos
# valores, vemos lo que pasa: es cierto que los casos que no corresponden a 'foreing_worker' son, muy mayoritariamente, de
# buen crédigo, pero es un porcentaje muy pequeño del total de casos.

alt.Chart(credit_g).mark_bar().encode(x = alt.X('class:N', title = None), 
                                         y = 'count(foreign_worker):Q', 
                                         color = alt.Color('class:N', legend = None),
                                         column = 'foreign_worker:O')

alt.Chart(...)

In [11]:
# Otras variables paracen más interesantes para predecir la calidad del crédito en muchos casos...

alt.Chart(credit_g).mark_bar().encode(x = alt.X('class:N', title = None), 
                                         y = 'count(checking_status):Q', 
                                         color = alt.Color('class:N', legend = None),
                                         column = 'checking_status:O')

alt.Chart(...)

In [12]:
alt.Chart(credit_g).mark_bar().encode(x = alt.X('class:N', title = None), 
                                         y = 'count(credit_history):Q', 
                                         color = alt.Color('class:N', legend = None),
                                         column = 'credit_history:O')

alt.Chart(...)

## 3. Modelos predictivos

In [13]:
# A partir de las observaciones del apartado anterior, podemos crear diversos modelos predictivos sencillos
# y calcular cómo de bien funcionan. Por ejemplo, un modelo podría consistir en hacer la media de los porcentajes
# de mal crédito que corresponden a los valores de las variables para un caso

In [14]:
def modelo1(caso):
    porcentajes = [mal_credito_por_valor[(mal_credito_por_valor['var'] == field) & 
                                         (mal_credito_por_valor['value'] == caso[field])]['percentage bad'].tolist()[0] \
                   for field in columnas_cat if field != 'class']
    return np.mean(porcentajes)

In [15]:
score1 = credit_g.apply(lambda x: modelo1(x), axis = 1)

In [16]:
# Podemos comprobar si este modelo tiene algún poder predictivo calculando el AUC ROC

from sklearn.metrics import roc_auc_score

roc_auc_score((credit_g['class'] == 'bad'), score1)

0.8008523809523809

In [17]:
# Un modelo más sencillo sería, por ejemplo, utilizar una cascada de criterios específicos
# (que es un método que, sorprendentemente, se utiliza en muchísimas ocasiones)

In [18]:
def modelo2(caso):
    if caso['duration'] >= 50 or caso['checking_status'] in(['\'<0\'', '\'0<=X<200\'']) or caso['credit_history'] in(['\'all paid\'', '\'no credits/all paid\'']):
        return 1
    else:
        return 0

In [19]:
score2 = credit_g.apply(lambda x: modelo2(x), axis = 1)

In [20]:
# Calculamos el poder predictivo de un modelo de este tipo y vemos que es más limitado

roc_auc_score((credit_g['class'] == 'bad'), score2)

0.6792857142857143

## 4. Coste de los errores cometidos por los modelos

In [21]:
# En el caso del modelo2, como el resultado es binario, simplemente se puede hacer una tabla de contingencia

tc_2 = pd.crosstab(score2, credit_g['class'])
tc_2

class,bad,good
row_0,,
0,54,377
1,246,323


In [22]:
# Podemos hacer una tabla con el coste de cada tipo de casos, para multiplicarla por la tabla de contigencia...

coste = pd.DataFrame([[5, 0], [0, 1]], columns=tc_2.columns, index=tc_2.index)
coste

class,bad,good
row_0,,
0,5,0
1,0,1


In [23]:
# El coste de usar el modelo 2 sería...

(tc_2 * coste).sum().sum()

593

In [24]:
# Para el modelo 1, como nos da una escala de certeza de la predicción, hay que seleccionar un umbral a partir del cual
# consideraríamos el caso como 'malo'

def tc_1(umbral):
    return pd.crosstab(score1 > umbral, credit_g['class'])

tc_1(0.30)

class,bad,good
row_0,,
False,57,460
True,243,240


In [25]:
def coste_1(umbral):
    return (tc_1(umbral) * coste).sum().sum()

coste_1(0.4)

1500.0

In [26]:
# Podemos generar una gráfica del coste que tendrían los errores de predicción, en función del umbral elegido...

x = np.linspace(0.2, 0.5, 100)
data = pd.DataFrame({
  'umbral': x,
  'coste': np.vectorize(coste_1)(x)
})

alt.Chart(data).mark_line().encode(x = 'umbral:Q', y = 'coste:Q')

alt.Chart(...)

In [27]:
# Se aprecia como el mínimo está alrededor de 0.29 y, naturalmente, como el modelo tiene mejor capacidad predictiva,
# el coste por errores de predicción es menor que en el caso del otro modelo.

coste_1(0.29)

518

A lo largo de toda esta unidad se ha estudiado como trabajar con datos y con modelos, pero únicamente se han desarrollado modelos triviales como los aquí presentados. Mucho del valor de la ciencia de datos proviene de la posibilidad de generar modelos más precisos, mediante técnicas de aprendizaje automático y de aprendizaje profundo. Todo eso se verá en unidades posteriores.